In [8]:
import pandas as pd
from pathlib import Path

## Loading Step

In [9]:
csv_paths = [x for x in Path("../output").glob("*.csv")]
dataframes = []
for x in csv_paths:
    county = x.name.strip(".csv").split("_")[0]
    temp = pd.read_csv(x.as_posix())
    temp["County"] = county
    dataframes.append(temp)
df = pd.concat(dataframes)
print(df.shape)
df.head()

(8586, 9)


,Indicator,Measure,Year,Jan-Mar (Provisional),Apr-Jun (Provisional),Jul-Sep (Provisional),Oct-Dec (Provisional),Year-to-Date (Provisional),County
0,Fatal Overdoses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Holmes
1,Opioid Overdose Deaths,Count,2016.0,1,0,1,1,3,Holmes
2,Drug Overdose Deaths,Count,2016.0,2,1,1,1,5,Holmes
3,Opioid Overdose Annual Age-Adjusted Death Rate,"Per 100,000 persons",2016.0,NaN,NaN,NaN,NaN,15.2,Holmes
4,Drug Overdose Annual Age-Adjusted Death Rate,"Per 100,000 persons",2016.0,NaN,NaN,NaN,NaN,25.6,Holmes


## Cleaning Step

In [16]:
df.drop_duplicates(inplace=True)
df.dropna(thresh=7, inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(5282, 10)


,Indicator,Measure,Year,Jan-Mar (Provisional),Apr-Jun (Provisional),Jul-Sep (Provisional),Oct-Dec (Provisional),Year-to-Date (Provisional),County,URL
0,Opioid Overdose Deaths,Count,2016.0,1,0,1,1,3,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...
1,Drug Overdose Deaths,Count,2016.0,2,1,1,1,5,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...
2,Emergency Medical Service Responses to a Suspe...,Count,2016.0,0,1,1,0,2,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...
3,Emergency Medical Service Responses to a Suspe...,Count,2016.0,7,6,10,2,25,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...
4,All Drug Non-fatal Overdose Emergency Departme...,Count,2016.0,14,12,7,11,44,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...


## Joining Step

... joins in the URL field

In [12]:
counties = {
	"Florida":      69,
	"Alachua":      1,
	"Baker":        2,
	"Bay":          3,
	"Bradford":     4,
	"Brevard":      5,
	"Broward":      6,
	"Calhoun":      7,
	"Charlotte":    8,
	"Citrus":       9,
	"Clay":         10,
	"Collier":      11,
	"Columbia":     12,
	"Miami-Dade":   13,
	"DeSoto":       14,
	"Dixie":        15,
	"Duval":        16,
	"Escambia":     17,
	"Flagler":      18,
	"Fanklin":      19,
	"Gadsden":      20,
	"Gilchrist":    21,
	"Glades":       22,
	"Gulf":         23,
	"Hamilton":     24,
	"Hardee":       25,
	"Hendry":       26,
	"Hernando":     27,
	"Highlands":    28,
	"Hillsborough": 29,
	"Holmes":       30,
	"India River":  31,
	"Jackson":      32,
	"Jefferson":    33,
	"Lafayette":    34,
	"Lake":         35,
	"Lee":          36,
	"Leon":         37,
	"Levy":         38,
	"Liberty":      39,
	"Madison":      40,
	"Manatee":      41,
	"Marion":       42,
	"Martin":       43,
	"Monroe":       44,
	"Nassau":       45,
	"Okaloosa":     46,
	"Okeechobee":   47,
	"Orange":       48,
	"Osceola":      49,
	"Palm Beach":   50,
	"Pasco":        51,
	"Pinellas":     52,
	"Polk":         53,
	"Putnam":       54,
	"St. Johns":    55,
	"St. Lucie":    56,
	"Santa Rosa":   57,
	"Sarasota":     58,
	"Seminole":     59,
	"Sumter":       60,
	"Suwannee":     61,
	"Taylor":       62,
	"Union":        63,
	"Volusia":      64,
	"Wakulla":      65,
	"Walton":       66,
	"Washington":   67,
}


In [19]:
url_lookups = dict()
for c in counties.keys():
    for y in {2015, 2016, 2017, 2018, 2019, 2020, 2021}:
        url_lookups[f"{c}_{y}"] = f"http://www.flhealthcharts.com/ChartsReports/rdPage.aspx?rdReport=SubstanceUseDashboard.SubstanceUseReport&ddlCounty={counties[c]}&ddlYear={y}&selTab=1"

df["URL"] = df.apply(lambda row: url_lookups[f"{row['County']}_{int(row['Year'])}"], axis=1)
df.head()

,Indicator,Measure,Year,Jan-Mar (Provisional),Apr-Jun (Provisional),Jul-Sep (Provisional),Oct-Dec (Provisional),Year-to-Date (Provisional),County,URL
0,Opioid Overdose Deaths,Count,2016.0,1,0,1,1,3,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...
1,Drug Overdose Deaths,Count,2016.0,2,1,1,1,5,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...
2,Emergency Medical Service Responses to a Suspe...,Count,2016.0,0,1,1,0,2,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...
3,Emergency Medical Service Responses to a Suspe...,Count,2016.0,7,6,10,2,25,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...
4,All Drug Non-fatal Overdose Emergency Departme...,Count,2016.0,14,12,7,11,44,Holmes,http://www.flhealthcharts.com/ChartsReports/rd...


## Write step

In [20]:
df.to_csv("../data/results.csv", index=False)